In [1]:
import secret_keys as key
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import pandas as pd

In [2]:
import tweepy

# Creating the authentication object
auth = tweepy.OAuthHandler(key.consumer_key, key.consumer_secret)
# Setting your access token and secret
auth.set_access_token(key.access_token, key.access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth,wait_on_rate_limit=True) 

In [3]:
search_words = '#corona OR #covid19 OR #covid OR #vaccine'
date_since = '2020-03-24'

In [4]:
tweets = tweepy.Cursor(api.search ,
                       q = search_words,
                       lang = 'en',
                       since = date_since
                      ).items(2000)

In [5]:
tweet_list = [[ tweet.geo, tweet.text, tweet.user.screen_name, tweet.user.location ] for tweet in tweets]

In [6]:
df_tweets = pd.DataFrame(tweet_list, columns = ['geo', 'tweet' , 'user_handle','location'])

In [7]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
geo            2 non-null object
tweet          2000 non-null object
user_handle    2000 non-null object
location       2000 non-null object
dtypes: object(4)
memory usage: 62.6+ KB


In [8]:
# https://towardsdatascience.com/another-twitter-sentiment-analysis-bb5b01ebad90
# Tweet data cleaning
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [9]:
df_tweets['cleaned_tweets'] = df_tweets.tweet.progress_apply(tweet_cleaner)

In [10]:
df_tweets.head()

,geo,tweet,user_handle,location,cleaned_tweets
0,None,RT @MelBala: The #Corona virus whatsapp line d...,__MJDr,Johannesburg.,rt the corona virus whatsapp line developed in...
1,None,"RT @ShashiTharoor: As an MP, I appeal to @PMOI...",ankushsharma200,Within 5 Oceans,rt as an mp i appeal to to divert the cr earma...
2,None,RT @EUErasmusPlus: Read the Erasmus Student Ne...,EU_Careers,Brussels,rt read the erasmus student network statement ...
3,None,RT @BWGovernment: Take heed of and follow heal...,RragoYaya,"Dikwididi, Kgatleng East, BW",rt take heed of and follow health professional...
4,None,"RT @ArewaTwiter: All Governors, Senators, Reps...",sagysagir,Nigeria,rt all governors senators reps or ministers et...


In [11]:
df_tweets['cleaned_tweets'] = df_tweets.cleaned_tweets.str.replace('rt', '')

In [12]:
df_tweets.head()

,geo,tweet,user_handle,location,cleaned_tweets
0,None,RT @MelBala: The #Corona virus whatsapp line d...,__MJDr,Johannesburg.,the corona virus whatsapp line developed in s...
1,None,"RT @ShashiTharoor: As an MP, I appeal to @PMOI...",ankushsharma200,Within 5 Oceans,as an mp i appeal to to dive the cr earmarked...
2,None,RT @EUErasmusPlus: Read the Erasmus Student Ne...,EU_Careers,Brussels,read the erasmus student network statement on...
3,None,RT @BWGovernment: Take heed of and follow heal...,RragoYaya,"Dikwididi, Kgatleng East, BW",take heed of and follow health professionals ...
4,None,"RT @ArewaTwiter: All Governors, Senators, Reps...",sagysagir,Nigeria,all governors senators reps or ministers etc ...


In [13]:
df_tweets.to_csv('../Data/Tweets_scraped_2000_26th_CORONA_COVID.csv',index=False)